In [1]:
!pip install bs4
#!pip install requestssoup = BeautifulSoup(data,"html5lib")
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=363a9153d9b13cf68e0d1a45db1378fa4cd2949985bc7f45c7d18888b7f5c9f9
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [2]:
#The below url contains html tables with data about world population.
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
# get the contents of the webpage in text format and store in a variable called data
data  = requests.get(url).text

In [4]:
soup = BeautifulSoup(data,"html5lib")  # create a soup object using the variable 'data'

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [34]:
df.shape

(103, 3)

In [6]:
df_postcode = df.assign(Neighborhood=df.Neighborhood.str.split(",")).explode('Neighborhood')

df_postcode.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Manor


In [7]:
url ='https://cocl.us/Geospatial_data'
locgeo_df  = pd.read_csv(url,index_col='Postal Code')
locgeo_df.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [8]:
toronto_data = df_postcode.join(locgeo_df, on='PostalCode')
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor,43.718518,-79.464763


In [9]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install folium
import folium # map rendering library

print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 6.9 MB/s  eta 0:00:01
Libraries imported.


In [10]:
#Filter only Toronto boroughs
toronto_borough_data = toronto_data.set_index('Borough').filter(like='Toronto', axis=0).reset_index(drop=True)
toronto_borough_data.head()

,PostalCode,Neighborhood,Latitude,Longitude
0,M5A,Regent Park,43.654260,-79.360636
1,M5A,Harbourfront,43.654260,-79.360636
2,M5B,Garden District,43.657162,-79.378937
3,M5B,Ryerson,43.657162,-79.378937
4,M5C,St. James Town,43.651494,-79.375418


In [11]:
neighborhood_latitude = toronto_borough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_borough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_borough_data.loc[0, 'Neighborhood'] # neighborhood name

In [12]:
CLIENT_ID = 'BQ4BZNHGBA5DIIO34LWZLUDJ3DEWTKNVT4BXC2UPPNW4DBQX' # your Foursquare ID
CLIENT_SECRET = 'TWRWDLTZZACDMKFNH2A4AFTNF3LWD12TI2DWDSMZVRBUOAAJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [13]:
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=BQ4BZNHGBA5DIIO34LWZLUDJ3DEWTKNVT4BXC2UPPNW4DBQX&client_secret=TWRWDLTZZACDMKFNH2A4AFTNF3LWD12TI2DWDSMZVRBUOAAJ&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '606a11fba1251b30a93b69da'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'display',
 

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
# type your answer here
toronto_venues = getNearbyVenues(names=toronto_borough_data['Neighborhood'],
                                   latitudes=toronto_borough_data['Latitude'],
                                   longitudes=toronto_borough_data['Longitude']
                                  )

Regent Park
 Harbourfront
Garden District
 Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond
 Adelaide
 King
Dufferin
 Dovercourt Village
The Danforth  East
Harbourfront East
 Union Station
 Toronto Islands
Little Portugal
 Trinity
The Danforth West
 Riverdale
Toronto Dominion Centre
 Design Exchange
Brockton
 Parkdale Village
 Exhibition Place
India Bazaar
 The Beaches West
Commerce Court
 Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park
 The Junction South
North Toronto West
The Annex
 North Midtown
 Yorkville
Parkdale
 Roncesvalles
Davisville
University of Toronto
 Harbord
Runnymede
 Swansea
Moore Park
 Summerhill East
Kensington Market
 Chinatown
 Grange Park
Summerhill West
 Rathnelly
 South Hill
 Forest Hill SE
 Deer Park
CN Tower
 King and Spadina
 Railway Lands
 Harbourfront West
 Bathurst Quay
 South Niagara
 Island airport
Rosedale
Enclave of M5E
St. James Town
 Cabbagetown
First Canadi

In [17]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,94,94,94,94,94,94
Bathurst Quay,16,16,16,16,16,16
Cabbagetown,44,44,44,44,44,44
Chinatown,64,64,64,64,64,64
Deer Park,15,15,15,15,15,15
Design Exchange,100,100,100,100,100,100
Dovercourt Village,15,15,15,15,15,15
Exhibition Place,24,24,24,24,24,24
Forest Hill SE,15,15,15,15,15,15


In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Adelaide,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.010638,0.000000,0.00,0.010638,0.010638,0.000000,0.010638,0.000000,0.000000,0.000000,0.0000,0.000000,0.010638,0.010638,0.010638,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.021277,0.000000,0.000000,0.010638,0.010638,0.000000,0.000000,0.010638,0.010638,0.021277,0.00,0.00,0.000000,0.053191,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031915,0.000000,0.117021,0.00000,0.00000,0.00,0.010638,0.000000,0.000000,0.021277,0.000000,0.021277,0.000000,0.000000,0.000000,0.010638,0.000000,0.031915,0.010638,0.000000,0.000000,0.

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
neighborhoods_venues_sorted.iloc[ind, 1:]
#print(toronto_grouped.head(5))
print('\n')
row = toronto_grouped.iloc[1, :]
print(row.head(5))
print('\n')
row_categories = row.iloc[1:]
print(row_categories.head(5))
print('\n')
row_categories_sorted = row_categories.sort_values(ascending=False)
print(row_categories_sorted.head(5))
print('\n')
row_categories_sorted.index.values[0:3]



Neighborhood           Bathurst Quay
Yoga Studio                        0
Airport                       0.0625
Airport Food Court            0.0625
Airport Lounge                 0.125
Name: 1, dtype: object


Yoga Studio                0
Airport               0.0625
Airport Food Court    0.0625
Airport Lounge         0.125
Airport Service       0.1875
Name: 1, dtype: object


Airport Service     0.1875
Airport Lounge       0.125
Airport Terminal     0.125
Plane               0.0625
Airport             0.0625
Name: 1, dtype: object




array(['Airport Service', 'Airport Lounge', 'Airport Terminal'],
      dtype=object)

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Clothing Store,Thai Restaurant,Concert Hall,Salad Place,Hotel
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Plane,Airport,Airport Food Court,Boutique,Harbor / Marina,Rental Car Location,Boat or Ferry
2,Cabbagetown,Coffee Shop,Italian Restaurant,Café,Bakery,Pizza Place,Restaurant,Pub,Snack Place,Indian Restaurant,Japanese Restaurant
3,Chinatown,Café,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Farmers Market,Arts & Crafts Store,Grocery Store,Bakery,Park
4,Deer Park,Coffee Shop,Pizza Place,Athletics & Sports,Vietnamese Restaurant,Liquor Store,Bagel Shop,Supermarket,Sushi Restaurant,American Restaurant,Light Rail Station
5,Design Exchange,Coffee Shop,Hotel,Café,Seafood Restaurant,Salad Place,Restaurant,Japanese Restaurant,Italian Restaurant,Concert Hall,Sporting Goods Shop
6,Dovercourt Village,Bakery,Pharmacy,Supermarket,Bank,Bar,Café,Middle Eastern Restaurant,Music Venue,Park,Coffee Shop
7,Exhibition Place,Café,Bakery,Breakfast Spot,Coffee Shop,Gym,Furniture / Home Store,Stadium,Restaurant,Intersection,Bar
8,Forest Hill SE,Coffee Shop,Pizza Place,Athletics & Sports,Vietnamese Restaurant,Liquor Store,Bagel Shop,Supermarket,Sushi Restaurant,American Restaurant,Light Rail Station
9,Grange Park,Café,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Farmers Market,Arts & Crafts Store,Grocery Store,Bakery,Park


In [23]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_borough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged

,PostalCode,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Regent Park,43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Shoe Store
1,M5A,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Shoe Store
2,M5B,Garden District,43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Ramen Restaurant,Lingerie Store
3,M5B,Ryerson,43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Ramen Restaurant,Lingerie Store
4,M5C,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Italian Restaurant,Restaurant,Bakery,Park,Cocktail Bar,Cosmetics Shop,Gym,Beer Bar
5,M4E,The Beaches,43.676357,-79.293031,0,Pub,Asian Restaurant,Health Food Store,Trail,Wings Joint,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
6,M5E,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Pharmacy,Restaurant,Farmers Market,Cheese Shop,Beer Bar,Department Store
7,M5G,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Thai Restaurant,Restaurant,Burger Joint,Yoga Studio
8,M6G,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Candy Store,Baby Store,Athletics & Sports,Restaurant,Coffee Shop
9,M5H,Richmond,43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Clothing Store,Thai Restaurant,Concert Hall,Salad Place,Hotel


In [25]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster - ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examined clusters to find dominant venues in each cluster

Cluster 1

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Regent Park,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Shoe Store
1,Harbourfront,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Shoe Store
2,Garden District,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Ramen Restaurant,Lingerie Store
3,Ryerson,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Ramen Restaurant,Lingerie Store
4,St. James Town,Coffee Shop,Café,Italian Restaurant,Restaurant,Bakery,Park,Cocktail Bar,Cosmetics Shop,Gym,Beer Bar
5,The Beaches,Pub,Asian Restaurant,Health Food Store,Trail,Wings Joint,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
6,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Pharmacy,Restaurant,Farmers Market,Cheese Shop,Beer Bar,Department Store
7,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Thai Restaurant,Restaurant,Burger Joint,Yoga Studio
8,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Candy Store,Baby Store,Athletics & Sports,Restaurant,Coffee Shop
9,Richmond,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Clothing Store,Thai Restaurant,Concert Hall,Salad Place,Hotel


Cluster 2

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Moore Park,Trail,Restaurant,Wings Joint,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
50,Summerhill East,Trail,Restaurant,Wings Joint,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Cluster 3

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,CN Tower,Airport Service,Airport Lounge,Airport Terminal,Plane,Airport,Airport Food Court,Boutique,Harbor / Marina,Rental Car Location,Boat or Ferry
60,King and Spadina,Airport Service,Airport Lounge,Airport Terminal,Plane,Airport,Airport Food Court,Boutique,Harbor / Marina,Rental Car Location,Boat or Ferry
61,Railway Lands,Airport Service,Airport Lounge,Airport Terminal,Plane,Airport,Airport Food Court,Boutique,Harbor / Marina,Rental Car Location,Boat or Ferry
62,Harbourfront West,Airport Service,Airport Lounge,Airport Terminal,Plane,Airport,Airport Food Court,Boutique,Harbor / Marina,Rental Car Location,Boat or Ferry
63,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Plane,Airport,Airport Food Court,Boutique,Harbor / Marina,Rental Car Location,Boat or Ferry
64,South Niagara,Airport Service,Airport Lounge,Airport Terminal,Plane,Airport,Airport Food Court,Boutique,Harbor / Marina,Rental Car Location,Boat or Ferry
65,Island airport,Airport Service,Airport Lounge,Airport Terminal,Plane,Airport,Airport Food Court,Boutique,Harbor / Marina,Rental Car Location,Boat or Ferry


Cluster 4

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Forest Hill North & West,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Wings Joint,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


Cluster 5

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,The Danforth East,Park,Convenience Store,Intersection,Wings Joint,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
32,Lawrence Park,Business Service,Park,Swim School,Bus Line,Wings Joint,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
66,Rosedale,Park,Playground,Trail,Wings Joint,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Added descriptive name to clusters based on dominant venues in that cluster

In [32]:
di = {0: "Restaurant", 1: "N/A", 2: "Airport", 3: "N/A", 4: "Park"}
#toronto_merged['Cluster Labels'].map(di) 
#toronto_merged['Cluster Labels Name'] = toronto_merged['Cluster Labels'].replace(di)
toronto_merged.insert(loc=5, column='Cluster Labels Name', value=toronto_merged['Cluster Labels'].replace(di))
toronto_merged

,PostalCode,Neighborhood,Latitude,Longitude,Cluster Labels,Cluster Labels Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Regent Park,43.654260,-79.360636,0,Restaurant,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Shoe Store
1,M5A,Harbourfront,43.654260,-79.360636,0,Restaurant,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Shoe Store
2,M5B,Garden District,43.657162,-79.378937,0,Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Ramen Restaurant,Lingerie Store
3,M5B,Ryerson,43.657162,-79.378937,0,Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Ramen Restaurant,Lingerie Store
4,M5C,St. James Town,43.651494,-79.375418,0,Restaurant,Coffee Shop,Café,Italian Restaurant,Restaurant,Bakery,Park,Cocktail Bar,Cosmetics Shop,Gym,Beer Bar
5,M4E,The Beaches,43.676357,-79.293031,0,Restaurant,Pub,Asian Restaurant,Health Food Store,Trail,Wings Joint,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
6,M5E,Berczy Park,43.644771,-79.373306,0,Restaurant,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Pharmacy,Restaurant,Farmers Market,Cheese Shop,Beer Bar,Department Store
7,M5G,Central Bay Street,43.657952,-79.387383,0,Restaurant,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Thai Restaurant,Restaurant,Burger Joint,Yoga Studio
8,M6G,Christie,43.669542,-79.422564,0,Restaurant,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Candy Store,Baby Store,Athletics & Sports,Restaurant,Coffee Shop
9,M5H,Richmond,43.650571,-79.384568,0,Restaurant,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Clothing Store,Thai Restaurant,Concert Hall,Salad Place,Hotel


### Recreated map with new descriptive cluster names

In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, cluster_name in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'], toronto_merged['Cluster Labels Name']):
    label = folium.Popup(str(poi) + ' - ' + str(cluster_name)+ ' Cluster', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters